In [1]:
import numpy as np
import pandas as pd 
import time
import requests
import lxml
from lxml.html import fromstring
from fake_useragent import UserAgent

ua = UserAgent()
url = "주소"
headers = {'User-Agent' : ua.random}

# 1. URL 수집

In [14]:
df = pd.DataFrame(columns=['url'])
for PAGE in range(1,132):
    
    # url 호출
    url = f'https://terms.naver.com/list.naver?cid=40942&categoryId=33147&so=st3.asc&viewType=&categoryType=&page={PAGE}'
    req = requests.get(url)
    doc  = fromstring(req.text)
    elements = doc.xpath('/html/body/div[1]/div[3]/div[1]/div[3]/ul/li/div/div[1]/strong/a[1]')
    
    # 정보수집 및 데이터프레임 추가
    values = ['https://terms.naver.com'+element.get('href') for element in elements]
    tmp = pd.DataFrame({'url':values})
    df = df.append(tmp, ignore_index=True)
    
    if PAGE % 10 == 0:
        print(f'{PAGE}페이지 완료!!')

10페이지 완료!!
20페이지 완료!!
30페이지 완료!!
40페이지 완료!!
50페이지 완료!!
60페이지 완료!!
70페이지 완료!!
80페이지 완료!!
90페이지 완료!!
100페이지 완료!!
110페이지 완료!!
120페이지 완료!!
130페이지 완료!!


# 2. 세부정보 수집
#### 산 이름, 한문, 요약, 위치, 봉우리 높이, 문화재, 본문 (이미지 제외) , 참조항목, 역참조항목, 카테고리

In [368]:
# 데이터프레임 초기화
df_final = pd.DataFrame()
t1 = time.time()


for idx in range(df.shape[0]):
    
    # url의 html을 호출
    url = df.loc[idx, 'url']
    req = requests.get(url)
    doc  = fromstring(req.text)

    # 정보 초기화
    MountainName, MountainName_Chinese, SUMMARY, LOCATION, HEIGHT, BONG, CULTURE, CONTENT, REFERENCE, REFERENCE_reverse, CATEGORY = ('', '', '', '', '', '', '', '', '', '', '')

    # 수집
    ### 산이름
    try:
        MountainName = doc.xpath('//*[@id="content"]/div[2]/div[1]/h2')[0].text
    except:
        MountainName = ''
    
    ### 산한자이름
    try:
        MountainName_Chinese = doc.xpath('//*[@id="content"]/div[2]/div[1]/p[3]/span')[0].text
    except:
        MountainName_Chinese = ''

    ### 요약
    try:
        SUMMARY = doc.xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/dl/text()')[1].replace('\n','').replace('\t','')
    except:
        SUMMARY = ''
        
################################################################################################
    ### 위치~문화재
    ##### 위치~문화재는 페이지별로 나타난 형식이 달라 다를 수 있어 두 가지 중 한 가지로 채워넣음
    for headline, element in zip(doc.xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/div/div/div/div/table/tbody/tr/th'), doc.xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/div/div/div/div/table/tbody/tr/td')):
        tmp_hd = ''.join(headline.itertext())
        tmp_el = ''.join(element.itertext()).replace('\n','').replace('\t','').strip()

        if tmp_hd == '위치':
            LOCATION = tmp_el

        elif tmp_hd == '높이':
            HEIGHT = tmp_el

        elif tmp_hd == '봉우리':
            BONG = tmp_el

        elif tmp_hd == '문화재':
            CULTURE = tmp_el
            CULTURE = {idx+1:culture.strip() for idx, culture in enumerate(CULTURE.split(','))}
            
        
    for headline, element in zip(doc.xpath('//*[@id="size_ct"]/div/div/div[2]/div/table/tbody/tr/th'), doc.xpath('//*[@id="size_ct"]/div/div/div[2]/div/table/tbody/tr/td')):
        tmp_hd = ''.join(headline.itertext())
        tmp_el = ''.join(element.itertext()).replace('\n','').replace('\t','').strip()

        if tmp_hd == '위치':
            LOCATION = tmp_el

        elif tmp_hd == '높이':
            HEIGHT = tmp_el

        elif tmp_hd == '봉우리':
            BONG = tmp_el

        elif tmp_hd == '문화재':
            CULTURE = tmp_el
            CULTURE = {idx+1:culture.strip() for idx, culture in enumerate(CULTURE.split(','))}
################################################################################################

    ### 본문
    try:
        CONTENT = ''.join(doc.xpath('//*[@id="size_ct"]/p')[0].itertext()).replace('\n','').replace('\t','')
    except:
        CONTENT = ''
    
    ### 참조항목 ~ 카테고리
    for element in doc.xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div'):
        tmp = ''.join(element.itertext())

        if tmp[:4] == '참조항목':
            REFERENCE = tmp[4:]
            REFERENCE = {idx+1:refer.strip() for idx, refer in enumerate(REFERENCE.split(','))}

        elif tmp[:5] == '역참조항목':
            REFERENCE_reverse = tmp[5:]
            REFERENCE_reverse = {idx+1:refer.strip() for idx, refer in enumerate(REFERENCE_reverse.split(','))}

        elif tmp.replace('\n',' ').replace('\t','')[1:5] == '카테고리':
            CATEGORY = tmp.replace('\n',' ').replace('\t','').strip()[4:].strip()
            CATEGORY = {idx+1:category for idx, category in enumerate(list(map(lambda x: x.strip(), CATEGORY.split('   '))))}

    ### 데이터프레임에 추가
    tmp = pd.DataFrame({'산이름':[MountainName],
                            '산이름_한자':[MountainName_Chinese],
                            '요약':[SUMMARY],
                            '위치':[LOCATION],
                            '높이':[HEIGHT],
                            '봉우리':[BONG],
                            '문화재':[CULTURE],
                            '본문':[CONTENT],
                            '참조항목':[REFERENCE],
                            '역참조항목':[REFERENCE_reverse],
                            '카테고리':[CATEGORY]})

    df_final = df_final.append(tmp, ignore_index=True)
    
    
    # log
    if (idx+1) % 100 == 0:
        print(f'=== {idx+1}개 수집 완료!! == 경과시간: {round(time.time()-t1)}초 ===')
        
        
print('크롤링 완료!')

=== 100개 수집 완료!! == 경과시간: 36초 ===
=== 200개 수집 완료!! == 경과시간: 74초 ===
=== 300개 수집 완료!! == 경과시간: 113초 ===
=== 400개 수집 완료!! == 경과시간: 149초 ===
=== 500개 수집 완료!! == 경과시간: 186초 ===
=== 600개 수집 완료!! == 경과시간: 223초 ===
=== 700개 수집 완료!! == 경과시간: 260초 ===
=== 800개 수집 완료!! == 경과시간: 299초 ===
=== 900개 수집 완료!! == 경과시간: 337초 ===
=== 1000개 수집 완료!! == 경과시간: 374초 ===
=== 1100개 수집 완료!! == 경과시간: 414초 ===
=== 1200개 수집 완료!! == 경과시간: 454초 ===
=== 1300개 수집 완료!! == 경과시간: 494초 ===
=== 1400개 수집 완료!! == 경과시간: 532초 ===
=== 1500개 수집 완료!! == 경과시간: 571초 ===
=== 1600개 수집 완료!! == 경과시간: 609초 ===
=== 1700개 수집 완료!! == 경과시간: 649초 ===
=== 1800개 수집 완료!! == 경과시간: 690초 ===
=== 1900개 수집 완료!! == 경과시간: 728초 ===
크롤링 완료!


In [369]:
df_final

,산이름,산이름_한자,요약,위치,높이,봉우리,문화재,본문,참조항목,역참조항목,카테고리
0,가덕산,加德山,경기도 가평군 북면과 강원도 춘천시 서면에 걸쳐 있는 산.,"경기 가평군 북면, 강원 춘천시 서면",858m,,,경기도 제1봉인 화악산의 지맥으로 모양새가 당당하고 곳곳에 깊은 골을 이루어 경계가...,"{1: '가평군', 2: '명지산', 3: '화악산', 4: '몽덕산', 5: '연...","{1: '석룡산', 2: '장락산'}","{1: '지리학 > 자연지리 > 산맥 및 산 > 한국', 2: '지역 > 아시아 >..."
1,가득봉,可得峰,강원 인제군 상남면(上南面)에 있는 고봉.,강원 인제군 상남면(上南面),"1,059 m",,,"높이 1,059 m. 가마산(可馬山) ·가칠봉(柯七峰) ·가리봉(加里峰) 등과 함께...","{1: '소양강', 2: '인제군', 3: '태백산맥', 4: '홍천군'}",,"{1: '지리학 > 자연지리 > 산맥 및 산 > 한국', 2: '지역 > 아시아 >..."
2,가라산,加羅山,경상남도 거제시 남부면 다대리에 있는 산.,경남 거제시,585m,가래봉,"{1: '거제 학동의 동백림 및 팔색조 번식지', 2: '거제도 연안의 아비 도래지...",높이 585m. 경상남도 남단 거제시의 최고봉으로 주봉은 가래봉이다. 산길에 서면 ...,"{1: '거제도', 2: '동부면', 3: '거제 학동리 동백나무 숲 및 팔색조 번...",{1: '탑포리'},"{1: '지리학 > 자연지리 > 산맥 및 산 > 한국', 2: '지역 > 아시아 >..."
3,가래산,加來山,"충청북도 청주시 상당구 낭성면 갈산리(葛山里), 인경리(仁景里)와 미원면 화창리(禾...",,,,,"높이 528m의 산으로 낭성면 갈산리(葛山里), 인경리(仁景里)와 미원면 화창리(禾...","{1: '화창리', 2: '인경리', 3: '갈산리'}",,"{1: '지역 > 아시아 > 한국 > 충청북도', 2: '지리학 > 자연지리 > 산..."
4,가련산,可連山,전라북도 전주시 덕진구 덕진동에 있는 산.,,,,,전주시 도심에 위치하며 높이는 106m이다. 전주시 주변은 동쪽 서쪽 남쪽으로 산지...,,,{1: '지리학 > 자연지리 > 산맥 및 산 > 한국'}
...,...,...,...,...,...,...,...,...,...,...,...
1947,흑석산,黑石山,"전라남도 해남군 계곡면과 영암군 학산면, 강진군 성전면에 걸쳐 있는 산.","전남 해남군 계곡면, 영암군 학산면, 강진군 성전면 지도",650m,깃대봉,,"전라남도 해남군 계곡면과 영암군 학산면, 강진군 성전면의 경계를 이루는 산으로 서쪽...","{1: '강진군', 2: '계곡면', 3: '성전면', 4: '영암군', 5: '학...",,"{1: '지역 > 아시아 > 한국 > 전라남도', 2: '지리학 > 자연지리 > 산..."
1948,흑성산,黑城山,충청남도 천안시 동남구 목천읍 남화리에 있는 산.,충남 천안시 동남구 목천읍 남화리,519m,,,높이는 519m이다. 충청남도 천안시 동남구 목천읍 남화리의 뒷산으로 차령산맥 줄기...,"{1: '독립기념관', 2: '병천면', 3: '천안시'}",,"{1: '지역 > 아시아 > 한국 > 충청남도', 2: '지리학 > 자연지리 > 산..."
1949,흙붉은오름,,제주특별자치도 제주시 아라동에 있는 오름.,제주특별자치도 제주시 아라동,"1,380.7m",,,토적악(土赤岳)이라고도 한다. 명칭은 오름을 구성하는 흙이 매우 붉은색을 띠는 데서...,"{1: '기생화산', 2: '한라산국립공원', 3: '아라동'}",,"{1: '지리학 > 자연지리 > 산맥 및 산 > 한국', 2: '지역 > 아시아 >..."
1950,흥림산,興霖山,경상북도 영양군 영양읍(英陽邑) ․청기면(靑杞面)․ 일월면(日月面)의 경계에 있는 ...,,,,,경상북도 영양군 영양읍(英陽邑) 청기면(靑杞面)과 일월면(日月面)의 경계에 있는 해...,"{1: '영양읍', 2: '일월면', 3: '청기면'}",,"{1: '지역 > 아시아 > 한국 > 경상북도', 2: '지리학 > 자연지리 > 산..."


In [371]:
df_final.to_excel('네이버_지식백과_산정보.xlsx', index=False)